In [1]:
import pandas as pd
import numpy as np
import tensorly as tl
from tensorly.decomposition import tucker
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA
import xgboost as xg
from scipy.fft import dct
from sklearn import metrics
import matplotlib.pyplot as plt
import sys

sys.path.insert(1,'D:\TMU_codes\Thesis_codes\codes')
# for line in sys.path:
#      print (line)
# from package.featureSelection_SICE import *
from package.loadDb import *
from package.featureSelection_TEMPORAL_PAGERANK import *

In [2]:
# import kaggle dataset
address = "..\database\kaggleDataFrame"

data_with_target= load_kaggle_dataframe(address+"\dataframes.csv")

data_with_target=data_with_target.iloc[:2000]

data_without_target = data_with_target.drop(['DEMAND'],axis=1)

print("data with target size = ",data_with_target.shape )

data with target size =  (2000, 11)


In [3]:
def optimum_window_size_tpr(datacut,win_list,validation_size,num_comp):
# datacut= data without validation samples
    n_rows,n_features=datacut.shape
    mse_per_win_dict={}
    for window_size in win_list:

        #input of xgboost
        X_datacut=datacut.iloc[:,:-1]
        Y_datacut=datacut.iloc[:,-1]

        #input of feature selection
        X_datacut_train=X_datacut.iloc[:n_rows-validation_size,:]
        print('optimum_window train',X_datacut_train.shape)
        # we apply feature selection on train set
        arr=create_graph_details(X_datacut_train,window_size)
        a=featureSelection_tpr(arr,n_features,num_comp)

        mse=xgboost_reg_error(X_datacut[X_datacut.columns[a][0]].values,Y_datacut,validation_size)
        mse_per_win_dict[window_size]=mse
    optimum_widowSize=min(mse_per_win_dict, key=mse_per_win_dict.get)
    return optimum_widowSize,mse_per_win_dict

In [4]:
data_without_target=data_with_target.iloc[:,:-1]
num_features=data_with_target.shape[1]

num_comp=4
# window_size=10
interval = 2000
test_size=200
win_list=[2,3,4,5,6,7,10,30]
validation_size=50

In [5]:
counter=0
pca_list=[]
dict_mse={}

while counter+interval <= data_without_target.shape[0]:


    datacut=data_with_target[counter:counter+interval]
    X_datacut=datacut.iloc[:,:-1]
    Y_datacut=datacut.iloc[:,-1]
    # print(counter,X_datacut.shape)
    X_pca=compute_pca(X_datacut,num_comp=num_comp)

    pca_mse=xgboost_reg_error(X_pca,Y_datacut,test_size)
    pca_list.append(pca_mse)##########################
    counter+=interval
dict_mse['PCA']=pca_list

    

method_list = []
counter=0

while counter+interval <= data_without_target.shape[0]:
    
    n_rows,n_features=datacut.shape

    datacut=data_with_target[counter:counter+interval]

    X_datacut=datacut.iloc[:,:-1]
    Y_datacut=datacut.iloc[:,-1]
    window_size,_=optimum_window_size_tpr(datacut.iloc[:n_rows-test_size],win_list,validation_size,num_comp)
    print("train",X_datacut.iloc[:n_rows-test_size].shape)
    arr=create_graph_details(X_datacut,window_size)
    a=featureSelection_tpr(arr,X_datacut.shape[1],num_comp)
    mse=xgboost_reg_error(X_datacut[X_datacut.columns[a][0]].values,Y_datacut,test_size)

    method_list.append(mse)
    #     print( "method_list",method_list)
    print(counter/interval)
    print("----------------------------------------------------------------------------")
    counter+=interval
# dict_mse["T-PR win={}".format(window_size)]=method_list
dict_mse["fs_temporal pr"]=method_list

dataframe = pd.DataFrame(dict_mse)
plt.rcParams["figure.figsize"] = [11,7]
dataframe.plot(kind="box",title="num_comp={}".format(num_comp))

NameError: name 'compute_pca' is not defined

In [ ]:
dataframe.plot(kind='line', marker='*', ylabel='RMSE',xlabel="dataframe number")


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
x_train, x_test, y_train, y_test = train_test_split(data_without_target, data_without_target.iloc[:,-1], test_size = 500,shuffle=False)
x_train=x_train.to_numpy()
x_train=x_train.reshape(x_train.shape[0],1,x_train.shape[1])
from sklearn.metrics import mean_absolute_error, mean_squared_error

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=( 1, 10)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=100, batch_size=1)
x_test=x_test.to_numpy()
x_test=x_test.reshape(x_test.shape[0],1,x_test.shape[1])

yhat = model.predict(x_test)
rmse=np.sqrt(metrics.mean_squared_error(y_test, yhat))


NameError: name 'Sequential' is not defined

In [ ]:
x_test=x_test.to_numpy()
x_test=x_test.reshape(x_test.shape[0],1,x_test.shape[1])

yhat = model.predict(x_test)
rmse=np.sqrt(metrics.mean_squared_error(y_test, yhat))


16/16 [==============================] - 1s 2ms/step


In [ ]:
rmse

27.205946120274056